In [6]:
import pandas as pd
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration
import operator

# Load the trained model and tokenizer
model_path = './best_t5_arithmetic_model'  # Path to the trained model
tokenizer = T5Tokenizer.from_pretrained(model_path)
model = T5ForConditionalGeneration.from_pretrained(model_path)

# Custom prefix evaluation function
def evaluate_prefix(expression):
    tokens = expression.split()
    stack = []
    
    ops = {
        '+': operator.add,
        '-': operator.sub,
        '*': operator.mul,
        '/': operator.truediv
    }
    
    for token in reversed(tokens):
        if token in ops:
            operand1 = stack.pop()
            operand2 = stack.pop()
            result = ops[token](operand1, operand2)
            stack.append(result)
        else:
            stack.append(float(token))
    
    return stack[0]

# Function to replace number placeholders with actual numbers
def replace_numbers_in_equation(equation, input_numbers):
    numbers = input_numbers.split()
    for i, number in enumerate(numbers):
        equation = equation.replace(f"number{i}", number)
    return equation

# Function to generate outputs from the test dataset
def generate_outputs(test_df):
    final_outputs = []
    
    model.eval()
    with torch.no_grad():
        for _, row in test_df.iterrows():
            description = row['Description']
            question = row['Question']
            input_numbers = row['Input Numbers']
            
            # Prepare the input for the model
            input_text = f"{description} {question}"
            input_ids = tokenizer(input_text, return_tensors='pt').input_ids
            
            # Generate the predicted equation
            outputs = model.generate(input_ids)
            predicted_equation = tokenizer.decode(outputs[0], skip_special_tokens=True)
            
            # Replace placeholders in the equation with actual numbers
            final_equation = replace_numbers_in_equation(predicted_equation, input_numbers)
            
            # Evaluate the final equation to get the result
            try:
                final_output = evaluate_prefix(final_equation)
            except Exception as e:
                final_output = None  # Handle evaluation error
            
            final_outputs.append(final_output)
    
    return final_outputs


# Load the test file
test_file_path = '../ArithOpsTestData1.xlsx'  # Provide the correct file path
test_df = pd.read_excel(test_file_path)

# Generate the final outputs for the test data
final_results = generate_outputs(test_df)

# Save the final output to an Excel file with no column header and only a single column of final outputs
output_file_path_single_column = 'FinalOutputsOnly.xlsx'

# Convert the final results to a DataFrame and save without headers
pd.DataFrame(final_results).to_excel(output_file_path_single_column, header=False, index=False)

print(f"Final outputs saved to {output_file_path_single_column}")

/home/trishitm/miniconda3/envs/dl-nlp/lib/python3.10/site-packages/transformers/generation/utils.py:1220: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Final outputs saved to FinalOutputsOnly.xlsx
